In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('drive')
os.chdir('MyDrive')
os.chdir('Coursera Project')

Mounted at /content/drive


In [ ]:
# Importing Necessary Libraries
import pandas as pd

In [ ]:
# Loading the transformed data
Company_names = ['Amazon', 'Apple', 'Google', 'Meta', 'Netflix']
company_dict = {}
for i in Company_names:
  company_dict[i]=pd.read_csv(i+'_df.csv').set_index('Date')

In [ ]:
# Importing Necessary Libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Setting up the batch size
batch_size = 64

In [ ]:
# Function to get the Train, Test, and Vaidation data
def construct_feature_target(df_in, f_len=3, batch_size = 64):
  df = df_in
  df_in = df_in.to_numpy()
  nbatch = len(df_in)//batch_size
  lendf = nbatch*batch_size
  X = df_in[:,:-1]
  y = df_in[:,-1:]
  X = np.reshape(X,(X.shape[0],X.shape[1],1))
  y = np.reshape(y,(y.shape[0],y.shape[1]))
  print(X.shape,y.shape)
  return df.index[0:int(0.8*nbatch)*batch_size], X[0:int(0.8*nbatch)*batch_size],y[0:int(0.8*nbatch)*batch_size], df.index[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size],X[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size],y[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size], df.index[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size], X[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size],y[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size]

In [ ]:
# Getting the Train, Test, and Vaidation data for each company and saving it to a dictionary
df_train = {}
X_train = {}
y_train = {}
df_val = {}
X_val = {}
y_val = {}
df_test = {}
X_test = {}
y_test = {}
sc = {}
for i in Company_names:
  df_train[i], X_train[i], y_train[i], df_val[i], X_val[i], y_val[i], df_test[i],  X_test[i], y_test[i]= construct_feature_target(company_dict[i], f_len=(company_dict[i].shape[1]-1), batch_size=batch_size)

(6464, 10, 1) (6464, 1)
(10624, 10, 1) (10624, 1)
(4608, 10, 1) (4608, 1)
(2688, 10, 1) (2688, 1)
(5184, 10, 1) (5184, 1)


In [ ]:
# Printing the shape Train, Test, and Vaidation data for each company
for i in Company_names:
  print(X_train[i].shape, y_train[i].shape,  X_val[i].shape, y_val[i].shape, X_test[i].shape, y_test[i].shape)

(5120, 10, 1) (5120, 1) (640, 10, 1) (640, 1) (704, 10, 1) (704, 1)
(8448, 10, 1) (8448, 1) (1088, 10, 1) (1088, 1) (1088, 10, 1) (1088, 1)
(3648, 10, 1) (3648, 1) (448, 10, 1) (448, 1) (512, 10, 1) (512, 1)
(2112, 10, 1) (2112, 1) (256, 10, 1) (256, 1) (320, 10, 1) (320, 1)
(4096, 10, 1) (4096, 1) (512, 10, 1) (512, 1) (576, 10, 1) (576, 1)


In [ ]:
# Importing Keras Libraries
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Importing Necessary Libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Loading the saved LSTM models
load = {}
for i in Company_names:
  load[i]=load_model(i+'_trained_model.h5')

In [ ]:
# Getting a dataframe for important performance indicators for the Statefull LSTM Regression model
metrics_list = ["Mean_Absolute_Error_Train", "Mean_Absolute_Error_Val", "Mean_Absolute_Error_Test",
                "Mean_Squared_Error_Train", "Mean_Squared_Error_Val","Mean_Squared_Error_Test",
                "R2_Score_Train", "R2_Score_Val", "R2_Score_Test",]
metrics = np.ones((5,9)).astype(np.float32)
for i in range(len(Company_names)):
  metrics[i][0] = mean_absolute_error(y_train[Company_names[i]],load[Company_names[i]].predict(X_train[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][3] = mean_squared_error(y_train[Company_names[i]],load[Company_names[i]].predict(X_train[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][6] = r2_score(y_train[Company_names[i]],load[Company_names[i]].predict(X_train[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][1] = mean_absolute_error(y_val[Company_names[i]],load[Company_names[i]].predict(X_val[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][4] = mean_squared_error(y_val[Company_names[i]],load[Company_names[i]].predict(X_val[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][7] = r2_score(y_val[Company_names[i]],load[Company_names[i]].predict(X_val[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][2] = mean_absolute_error(y_test[Company_names[i]],load[Company_names[i]].predict(X_test[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][5] = mean_squared_error(y_test[Company_names[i]],load[Company_names[i]].predict(X_test[Company_names[i]], batch_size = 64, verbose=0))
  metrics[i][8] = r2_score(y_test[Company_names[i]],load[Company_names[i]].predict(X_test[Company_names[i]], batch_size = 64, verbose=0))

In [ ]:
# Printing the dataframe for important performance indicators for the Statefull LSTM Regression model
df = pd.DataFrame(metrics, columns=metrics_list)
df["Company"] = Company_names
df = df.set_index("Company")
df

,Mean_Absolute_Error_Train,Mean_Absolute_Error_Val,Mean_Absolute_Error_Test,Mean_Squared_Error_Train,Mean_Squared_Error_Val,Mean_Squared_Error_Test,R2_Score_Train,R2_Score_Val,R2_Score_Test
Company,,,,,,,,,
Amazon,0.008395,0.007295,0.028893,0.000073,0.000103,0.001199,0.982167,0.979214,0.943841
Apple,0.008801,0.006974,0.018277,0.000078,0.000055,0.000628,0.914197,0.976146,0.989179
Google,0.007533,0.007606,0.028573,0.000064,0.000113,0.001239,0.994125,0.983882,0.918329
Meta,0.007951,0.030599,0.015897,0.000109,0.001318,0.000575,0.996665,0.870148,0.981586
Netflix,0.008531,0.011462,0.019239,0.000081,0.000249,0.000709,0.994139,0.980316,0.985008


In [ ]:
# Saving the dataframe for important performance indicators for the Statefull LSTM Regression model
df.to_csv("Model_Evaluation.csv")